# Μέρος Δ
Ταξινόμηση με Random Forest
Εκπαιδεύουμε τον ταξινομητή με το training set datasetTV.csv και στη συνέχεια εφαρμόζουμε τον αλγόριθμο ταξινόμησης στα δεδομένα datasetTest.csv.

## 1. Φόρτωση και προεπεξεργασία των δεδομένων
- Φορτώνουμε το training dataset (`datasetTV.csv`) και το χωρίζουμε σε features και labels.
- Φορτώνουμε το test dataset (`datasetTest.csv`).
- Ελέγχουμε ότι τα ονόματα των στηλών στο test dataset ταιριάζουν με τα features του training dataset.
- Κανονικοποιούμε τα δεδομένα με τη μέθοδο `StandardScaler`.

In [2]:

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load datasets
dataset_tv_path = "datasetTV.csv"
dataset_test_path = "datasetTest.csv"

train_data = pd.read_csv(dataset_tv_path)
test_data = pd.read_csv(dataset_test_path)

# Split into features and labels
X_train = train_data.iloc[:, :-1]  # All features except the last column
y_train = train_data.iloc[:, -1]   # Last column as the label

# Rename columns of test data to match training data
X_test = test_data
X_test.columns = X_train.columns

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 2. Εκπαίδευση του εκτιμητή με τη μέθοδο Random Forest
Εκπαιδεύουμε τον ταξινομητή `RandomForestClassifier` χρησιμοποιώντας hyperparameter tuning με τη μέθοδο `GridSearchCV` που θα επιλέξει τις καλύτερες hyperparameters.
Αξιολογούμε την απόοδση του μοντέλου για κάθε συνδυασμό με cross-validation και επιλέγουμε το καλύτερο μοντέλο.

In [3]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Train a Random Forest Classifier with hyperparameter tuning
rf_clf = RandomForestClassifier(random_state=42)
params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(rf_clf, params, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best model
best_rf = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Evaluate on the training data
y_train_pred = best_rf.predict(X_train_scaled)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")


Best Parameters: {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 150}
Training Accuracy: 1.0000


## 3. Προβλέψεις στο Test Set
Έχοντας εκπαιδεύσει το μοντέλο, προβλέπουμε τα labels για το test dataset και σώζουμε τις προβλέψεις σε ένα NumPy array.

In [6]:

# Predict on the test set
y_test_pred = best_rf.predict(X_test_scaled)

# Save predictions to a numpy file
output_path = "labels68.npy"
np.save(output_path, y_test_pred)

print(f"Predictions saved to {output_path}")
np.load('labels68.npy')

Predictions saved to labels68.npy


array([4, 5, 2, ..., 3, 4, 1], shape=(6954,))